# Section 8.2: Time Matrix Multiply

Adapted from: "Guide to Fortran 2008 Programming" by Walter S. Brainerd (Springer 2015)

## Program to demonstrate the performance improvement of using matmul over do loops.

```fortran
program time_matrix_multiply

    ! Compare times of the matmul intrinsic cs DO loops

    implicit none
    integer, parameter :: n = 10
    real, dimension (n, n) :: a, b, c1, c2
    character(len=8) :: date
    real :: start_time_1, stop_time_1, start_time_2, stop_time_2
    real :: total_time_1, total_time_2
    integer :: i, j, k
    character(len=*), parameter :: form = "(t2, a, f0.10, a)"

    ! Get date to print on report

    call date_and_time(date = date)

    print *, "Timing report dated: " // date(1:4) &
        // "-" // date(5:6) // "-" // date(7:8)

    call random_seed()
    call random_number(a)
    call random_number(b)
    call cpu_time(start_time_1)
   
    ! Lines below added for effect
    ! The matrices of random values will be printed to the screen
    write (*, "(a)") "Matrix A"
    write (*, "(10f10.3)") (a(i,:), i=1,n)

    print *
    write (*, "(a)") "Matrix B"
    write (*, "(10f10.3)") (b(i,:), i=1,n)

    c1 = 0

    do k = 1, n
        do j = 1, n
            do i = 1, n
                c1(i, j) = c1(i, j) + a(i, k) * b(k, j)
            end do
        end do
    end do

    call cpu_time(stop_time_1)

    total_time_1 = stop_time_1 - start_time_1

    print *

    write (*, "(a)") "Matrix C1 is AxB using loops."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    call cpu_time(start_time_2)
    c2 = matmul(a, b)
    call cpu_time(stop_time_2)

    total_time_2 = stop_time_2 - start_time_2

    print *

    write (*, "(a)") "Matrix C2 is AxB using matmul."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    print *

    write (*, form) "Time of Do loop version is: ", total_time_1, " seconds."
    write (*, form) "Time of matmul version is: ", total_time_2, " seconds."

    print *

    if (any(abs(c1-c2) > 1.0e-4)) then
        write (*,  "(a)") "There are significantly different values between the matrices."
    else
        write (*,  "(a)") "The results are approximately the same."
    end if

    print *
    write (*, "(a, f10.3, a)") "The speedup ratio is: ", total_time_1/total_time_2, "x"

end program time_matrix_multiply
```

The above program is compiled and run using Fortran Package Manager (fpm):

In [1]:
import os
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_8_2_Time_Matrix_Multiply"

In [3]:
os.chdir(code_dir)

In [4]:
build_status = os.system("fpm build 2>/dev/null")

In [5]:
exec_status = os.system("fpm run 2>/dev/null")

 Timing report dated: 2023-12-26
Matrix A
     0.826     0.448     0.546     0.144     0.603     0.849     0.846     0.219     0.846     0.739
     0.632     0.084     0.131     0.831     0.200     0.475     0.203     0.888     0.657     0.233
     0.411     0.608     0.760     0.897     0.168     0.972     0.645     0.256     0.273     0.205
     0.334     0.336     0.374     0.596     0.145     0.710     0.388     0.228     0.667     0.258
     0.950     0.955     0.299     0.765     0.102     0.008     0.697     0.787     0.956     0.870
     0.431     0.481     0.756     0.338     0.040     0.535     0.220     0.938     0.463     0.284
     0.456     0.401     0.751     0.462     0.479     0.059     0.202     0.961     0.231     0.616
     0.980     0.113     0.497     0.692     0.737     0.252     0.040     0.882     0.086     0.918
     0.165     0.311     0.723     0.574     0.615     0.201     0.756     0.159     0.348     0.385
     0.001     0.049     0.936     0.031     0.01